## Abstract Attributes

Not working yet.

In [120]:
def abstractattrs(*attrs):
    def wrapper(cls):
        
        def method(cls, *args, **kwargs):
            super(cls, cls).__init_subclass__(*args, **kwargs)
            for attr in attrs:
                if not hasattr(cls, attr):
                    raise RuntimeError(f'Class must have attribute {attr}.') 
                    
        setattr(cls, '__init_subclass__', classmethod(method))
        return cls
    return wrapper              

In [121]:
@abstractattrs('name', 'age')
class Base:
    def __init__(self, age=7):
        self.age = age 

In [122]:
class Child(Base):
    a = 'a'

RecursionError: maximum recursion depth exceeded while calling a Python object

In [118]:
class AbstractAttrs:
    
    def __init__(self, *args):
        print(args)
        self.args = list(args)
    
    @classmethod
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
#         if not hasattr(cls, f'_{cls.__name__}__attrs'):
#             print('no attrs')
        print('init subclass', AbstractAttrs._args())
        for arg in AbstractAttrs.args:
            assert hasattr(cls, arg), \
                f'{cls.__name__} must have class variable {arg}.'

In [119]:
class Callback(AbstractAttrs):
    pass

TypeError: _args() missing 1 required positional argument: 'self'

In [112]:
c = Callback('order', 'is_val')

('order', 'is_val')


In [113]:
class NewCallback(Callback):
    
    def __init__(self, a, order=3):
        self.a = a
        self.order = order

pre super
NewCallback
no attrs


In [107]:
nc = NewCallback('a', 'my_order')

In [108]:
nc

In [109]:
nc._attrs

['order', 'val_loop']

In [549]:
class AbstractAttrs(type):

    def __new__(cls, name, bases, methods, **meta_kwargs):
        print('new meta kwargs', meta_kwargs)
        class_ = type.__new__(cls, name, bases, methods)
        class_vars = meta_kwargs.get('class_vars', [])
        inst_vars = meta_kwargs.get('inst_vars', [])
        
        def __init_subclass__(cls, **kwargs):
            print('init subclass kwargs', kwargs)
            super().__init_subclass__(**kwargs)
            for attr in class_vars:
                assert hasattr(cls, attr), \
                    f'{cls} must have class variable {attr}.'
        
        # Without this check, we'd overwrite these attributes when subclasses
        # are constructed.
        if class_vars or inst_vars:
            setattr(class_, '__init_subclass__', 
                    classmethod(__init_subclass__))
            class_._is_parent = True
            class_.class_vars = class_vars
            class_.inst_vars = inst_vars
#             setattr(class_, '_is_parent', True)
#             setattr(class_, 'class_vars', class_vars)
#             setattr(class_, 'inst_vars', inst_vars)
        else:
#             setattr(class_, '_is_parent', False)
            class_._is_parent = False
        return class_
    
    def __init__(cls, name, bases, args, **kwargs):
        print('init')
        print(cls, name, bases, args, kwargs)
#         return cls.__init__(args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('in call', 'args=', args, 'kwargs=', kwargs)
        inst = cls.__new__(cls, *args, **kwargs)
        if isinstance(inst, cls):
            inst.__init__(*args, **kwargs)
            print('in isinstance, instvars=', inst.inst_vars)
            if not cls._is_parent:
                for var in inst.inst_vars:
                    assert hasattr(inst, var), f'Missing instance var {var}.'
        return inst

In [564]:
class Parent(metaclass=AbstractAttrs, inst_vars=['name'],
             class_vars=['order', 'is_val']):
    pass

new meta kwargs {'inst_vars': ['name'], 'class_vars': ['order', 'is_val']}
init
<class '__main__.Parent'> Parent () {'__module__': '__main__', '__qualname__': 'Parent'} {'inst_vars': ['name'], 'class_vars': ['order', 'is_val']}


In [551]:
p = Parent()

in call args= () kwargs= {}
in isinstance, instvars= ['name']


In [552]:
p.class_vars, p.inst_vars

(['order', 'is_val'], ['name'])

In [553]:
p._is_parent

True

In [554]:
class Child(Parent):
    
    order = 33
    is_val = True
    
    def __init__(self, a):
        self.a = a
        print('child init')

new meta kwargs {}
init subclass kwargs {}
init
<class '__main__.Child'> Child (<class '__main__.Parent'>,) {'__module__': '__main__', '__qualname__': 'Child', 'order': 33, 'is_val': True, '__init__': <function Child.__init__ at 0x1a14d0b950>} {}


In [555]:
c = Child(4)
c

in call args= (4,) kwargs= {}
child init
in isinstance, instvars= ['name']


AssertionError: Missing instance var name.

In [556]:
c.a, c.class_vars, c.inst_vars

(4, ['order', 'is_val'], ['name'])

In [557]:
c._is_parent

False

In [558]:
c.test

99

In [559]:
class BadChild(Parent):
    
    order = 1
    
    def __init__(self, b, is_val):
        self.b = b
        self.is_val = is_val

new meta kwargs {}
init subclass kwargs {}


AssertionError: <class '__main__.BadChild'> must have class variable is_val.

In [561]:
class GoodChild(Parent):
    
    order = 100
    is_val = False
    
    def __init__(self, name):
        self.name = name
        
    def walk(self):
        return self.order, self.is_val, self.name

new meta kwargs {}
init subclass kwargs {}
init
<class '__main__.GoodChild'> GoodChild (<class '__main__.Parent'>,) {'__module__': '__main__', '__qualname__': 'GoodChild', 'order': 100, 'is_val': False, '__init__': <function GoodChild.__init__ at 0x1a14d0b378>, 'walk': <function GoodChild.walk at 0x1a14d0bf28>} {}


In [562]:
ian = GoodChild('Ian')

in call args= ('Ian',) kwargs= {}
in isinstance, instvars= ['name']


In [563]:
ian.walk()

(100, False, 'Ian')

## LSH

In [30]:
from datasketch import lsh, MinHashLSH, MinHash, MinHashLSHForest

from htools import *

In [2]:
set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)

In [3]:
def ngrams(word, n=3, step=1, drop_last=False):
    """To get non-overlapping sequences, pass in same value for `step` as `n`."""
    stop = max(1, step+len(word)-n)
    ngrams_ = []
    for i in range(0, stop, step):
        ngrams_.append(word[i:i+n])
    if drop_last and len(ngrams_[-1]) < n: ngrams_ = ngrams_[:-1]
    return ngrams_

In [35]:
w = 'dinosaur'
tw = 'I'
ws = ['dino', 'rhino', 'dog', 'cat', 'lion', 'tyrannosaurus rex',
      'tiger', 'owl', 'rhinocerous', 'kino']
s = 'The mailman was running late today, possibly due to the heavy '\
    'snowfall from the previous night.'

In [6]:
ngrams(w)

['din', 'ino', 'nos', 'osa', 'sau', 'aur']

In [7]:
ngrams(w, 2)

['di', 'in', 'no', 'os', 'sa', 'au', 'ur']

In [9]:
ngrams(w, 3, 3)

['din', 'osa', 'ur']

In [12]:
ngrams(w, 3, 3, True)

['din', 'osa']

In [37]:
ngrams(s)

['The',
 'he ',
 'e m',
 ' ma',
 'mai',
 'ail',
 'ilm',
 'lma',
 'man',
 'an ',
 'n w',
 ' wa',
 'was',
 'as ',
 's r',
 ' ru',
 'run',
 'unn',
 'nni',
 'nin',
 'ing',
 'ng ',
 'g l',
 ' la',
 'lat',
 'ate',
 'te ',
 'e t',
 ' to',
 'tod',
 'oda',
 'day',
 'ay,',
 'y, ',
 ', p',
 ' po',
 'pos',
 'oss',
 'ssi',
 'sib',
 'ibl',
 'bly',
 'ly ',
 'y d',
 ' du',
 'due',
 'ue ',
 'e t',
 ' to',
 'to ',
 'o t',
 ' th',
 'the',
 'he ',
 'e h',
 ' he',
 'hea',
 'eav',
 'avy',
 'vy ',
 'y s',
 ' sn',
 'sno',
 'now',
 'owf',
 'wfa',
 'fal',
 'all',
 'll ',
 'l f',
 ' fr',
 'fro',
 'rom',
 'om ',
 'm t',
 ' th',
 'the',
 'he ',
 'e p',
 ' pr',
 'pre',
 'rev',
 'evi',
 'vio',
 'iou',
 'ous',
 'us ',
 's n',
 ' ni',
 'nig',
 'igh',
 'ght',
 'ht.']

In [13]:
def lsh_hash_word(word, num_perm=128, **ngram_kwargs):
    mhash = MinHash(num_perm=num_perm)
    for ng in ngrams(word, **ngram_kwargs):
        mhash.update(ng.encode('utf8'))
    return mhash

In [16]:
m = lsh_hash_word(w)

In [22]:
forest = MinHashLSHForest(num_perm=128)

for i, word in enumerate(ws):
    forest.add(i, lsh_hash_word(word))
forest.index()

In [24]:
forest.query(lsh_hash_word(w), 3)

[8, 0, 5]

In [26]:
ws

['dino',
 'rhino',
 'dog',
 'cat',
 'lion',
 'tyrannosaurus rex',
 'tiger',
 'owl',
 'rhinocerous',
 'kino']

In [28]:
idx = forest.query(lsh_hash_word(w), 3)
idx

[8, 0, 5]

In [32]:
select(ws, idx)

['dino', 'tyrannosaurus rex', 'rhinocerous']